In [1]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 55.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.9 MB/s eta 0:00:00:00:0100:01
  Attempting un

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [4]:
df = pd.read_csv("/kaggle/input/sample-text/sample_text.csv")
df.shape

(8, 2)

In [5]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


Create source embeddings for the text column

In [6]:
from sentence_transformers import SentenceTransformer

2025-09-09 10:11:49.138081: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757412709.349741      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757412709.409763      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [20]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
vectors.shape

(8, 768)

In [22]:
vectors

array([[-0.00247395,  0.03626722, -0.05290459, ..., -0.09152356,
        -0.03970001, -0.04330489],
       [-0.03357267,  0.00980519, -0.03250129, ..., -0.05165466,
         0.02245887, -0.03156182],
       [-0.01865322, -0.04051318, -0.01235387, ...,  0.00610586,
        -0.07179645,  0.02773851],
       ...,
       [-0.00066458,  0.04252127, -0.05645508, ...,  0.0131547 ,
        -0.03183567, -0.04357665],
       [-0.03317153,  0.03252455, -0.02484838, ...,  0.0117442 ,
         0.05747124,  0.00571023],
       [-0.00166395,  0.00413828, -0.04597083, ...,  0.02008527,
         0.05656243, -0.00161596]], dtype=float32)

In [23]:
dim = vectors.shape[1]
dim

768

Build a FAISS Index for vectors

In [24]:
import faiss

index = faiss.IndexFlatL2(dim)

Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [25]:
index.add(vectors)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7b91240e8360> >

Encode search text using same encorder and normalize the output vector

In [37]:
search_query_0 = "I want to buy a polo t-shirt"
search_query_1 = "looking for places to visit during the holidays"
search_query_2 = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query_0)
vec1 = encoder.encode(search_query_1)
vec2 = encoder.encode(search_query_2)
vec.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(768,)

In [40]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec1 = np.array(vec1).reshape(1,-1)
svec2 = np.array(vec2).reshape(1,-1)
svec.shape

(1, 768)

Search for similar vector in the FAISS index created

In [44]:
#index.search(svec, k=2)
index.search(svec1, k=2)
#index.search(svec2, k=2)

(array([[0.92737275, 1.1601744 ]], dtype=float32), array([[6, 7]]))

In [45]:
distances, I = index.search(svec1, k=2)
I

array([[6, 7]])

In [46]:
df.loc[[6, 7]]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [47]:
I.tolist()

[[6, 7]]

In [48]:
row_indices = I.tolist()[0]
row_indices

[6, 7]

In [49]:
df.loc[row_indices]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel
